### Fake news classifier using lstm



In [104]:
import pandas as pd

In [105]:
df=pd.read_csv('fake_news.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [106]:
df=df.dropna()

In [107]:
## Get the independent features
X = df.drop('label', axis=1)

In [108]:
## Get the Dependent features
y = df['label']

In [109]:
X.shape

(18285, 4)

In [110]:
y.shape

(18285,)

In [111]:
import tensorflow as tf
tf.__version__

'2.20.0'

In [112]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [113]:
### Vocabulary size
voc_size = 5000

#### Onehot representation

In [114]:
messages=X.copy()

In [115]:
messages.reset_index(inplace=True)


In [116]:
import nltk
import re
from nltk.corpus import stopwords

In [117]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jitendrakumar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [118]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus=[]
for i in range(0, len(messages)):
    review =re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)




In [119]:
corpus


['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [120]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]

In [121]:
onehot_repr


[[4010, 1414, 3397, 2309, 1757, 1502, 110, 1946, 3811, 1111],
 [2907, 2246, 170, 2445, 659, 3556, 3231],
 [283, 3260, 1041, 2887],
 [4993, 4007, 1840, 2391, 1435, 760],
 [209, 659, 1297, 1435, 2384, 2217, 659, 1601, 2301, 8],
 [4216,
  2625,
  2711,
  2976,
  2975,
  865,
  484,
  2337,
  3490,
  3297,
  4440,
  3153,
  1614,
  4984,
  3231],
 [242, 610, 1747, 1638, 1857, 3469, 4641, 3143, 2437, 2909, 3604],
 [985, 3227, 3211, 2039, 1080, 4712, 865, 2729, 2437, 2909, 3604],
 [4130, 3928, 1979, 4584, 4617, 2271, 1640, 2979, 865, 1451],
 [3175, 2059, 4238, 4726, 2073, 2054, 3636, 3110],
 [3300, 1222, 1822, 2738, 3232, 4646, 4634, 4383, 2072, 4321, 642],
 [2391, 4807, 1757, 2271, 865, 1080],
 [676, 2128, 2315, 3578, 138, 3857, 1944, 1281, 4667],
 [2902, 4378, 3201, 3818, 3242, 3755, 1156, 2437, 2909, 3604],
 [4772, 985, 2937, 2077, 1389, 2437, 2909, 3604],
 [3598, 2184, 306, 243, 2536, 2374, 218, 883, 4743, 4334],
 [2388, 4005, 2246],
 [2002, 2414, 3606, 2427, 865, 3136, 3580, 3231],
 [40

### Embedding representation

In [122]:
sent_length=20
embedded_docs = pad_sequences(onehot_repr, maxlen=sent_length, padding='pre')
embedded_docs


array([[   0,    0,    0, ..., 1946, 3811, 1111],
       [   0,    0,    0, ...,  659, 3556, 3231],
       [   0,    0,    0, ..., 3260, 1041, 2887],
       ...,
       [   0,    0,    0, ..., 2437, 2909, 3604],
       [   0,    0,    0, ..., 3576, 1464, 2503],
       [   0,    0,    0, ..., 2801, 3091,  926]],
      shape=(18285, 20), dtype=int32)

In [123]:
embedded_docs[0]


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4010,
       1414, 3397, 2309, 1757, 1502,  110, 1946, 3811, 1111], dtype=int32)

In [124]:
## Creating model
embedded_vector_features =40
model =Sequential()
model.add(Embedding(voc_size, embedded_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/Users/jitendrakumar/development/python_projects/nlp/venv/lib/python3.13/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [125]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [126]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)



#### Model Training

In [128]:
### Finally Training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8659 - loss: 0.2961 - val_accuracy: 0.9122 - val_loss: 0.1971
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9435 - loss: 0.1449 - val_accuracy: 0.9218 - val_loss: 0.2164
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9614 - loss: 0.1026 - val_accuracy: 0.9135 - val_loss: 0.2167
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9736 - loss: 0.0770 - val_accuracy: 0.9099 - val_loss: 0.2624
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9793 - loss: 0.0612 - val_accuracy: 0.9102 - val_loss: 0.2663
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9853 - loss: 0.0462 - val_accuracy: 0.9110 - val_loss: 0.3005
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9889 - loss: 0.0342 - val_accuracy: 0.9046 - val_loss: 0.3560
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9916 - loss: 0.0266 - val_accuracy: 0.

### Adding Dropout

In [129]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dropout(0.3)) 
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [137]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [144]:
from sklearn.metrics import confusion_matrix
y_pred_labels = (y_pred).astype(int)  # Convert probabilities to 0/1
confusion_matrix(y_test, y_pred_labels)

array([[3419,    0],
       [2616,    0]])

In [145]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_labels)

0.5665285832642917